In [160]:
#!pip install pillow
#!pip install pytesseract
#!pip install opencv-python


In [161]:
from PIL import Image, ImageFont, ImageDraw
import cv2 
import pytesseract
import numpy as np
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [162]:
my_image = Image.open(r"C:\Users\prateek\Desktop\Ayoo docs\Code\paperless\forms\cs_inventory.jpg")

In [163]:
#Below ttf file is openly avilable font in human handwriting.
# We can create custom ttf files using https://pypi.org/project/handwrite/

title_font = ImageFont.truetype(r'C:\Users\prateek\Desktop\Ayoo docs\Code\paperless\fonts\GrapeNuts-Regular.ttf', size =100)

In [164]:
# #ocr with tesseract - Will use this pipeline later to preprocess images. Not using curently

img = cv2.imread(r"C:\Users\prateek\Desktop\Ayoo docs\Code\paperless\forms\cs_inventory.jpg")

# # get grayscale image
# def get_grayscale(image):
#     return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # noise removal
# def remove_noise(image):
#     return cv2.medianBlur(image,5)
 
# #thresholding
# def thresholding(image):
#     return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# #dilation
# def dilate(image):
#     kernel = np.ones((5,5),np.uint8)
#     return cv2.dilate(image, kernel, iterations = 1)
    
# #erosion
# def erode(image):
#     kernel = np.ones((5,5),np.uint8)
#     return cv2.erode(image, kernel, iterations = 1)

# #opening - erosion followed by dilation
# def opening(image):
#     kernel = np.ones((5,5),np.uint8)
#     return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# #canny edge detection
# def canny(image):
#     return cv2.Canny(image, 100, 200)

# #skew correction
# def deskew(image):
#     coords = np.column_stack(np.where(image > 0))
#     angle = cv2.minAreaRect(coords)[-1]
#     if angle < -45:
#         angle = -(90 + angle)
#     else:
#         angle = -angle
#     (h, w) = image.shape[:2]
#     center = (w // 2, h // 2)
#     M = cv2.getRotationMatrix2D(center, angle, 1.0)
#     rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
#     return rotated


In [165]:
from pytesseract import Output


d = pytesseract.image_to_data(img, output_type=Output.DICT)
print(d.keys())

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


In [166]:
for i in range(len(d['text'])):
    print(i, d['text'][i])

0 
1 
2 
3 
4 XYZ
5 pharmacy
6 
7 Controlled
8 Substance
9 Inventory
10 
11 
12 
13 Report
14 Contents
15 (circle
16 one):
17 
18 CIll-V
19 PSE/E*
20 
21 
22 
23 *if
24 required
25 by
26 your
27 state
28 
29 
30 
31  
32 
33 
34 
35 XYZ/pharmacy
36 Store
37 #
38 
39 Address:
40 
41 City.
42 OC“
43 CS
44 Statte:
45 Zip:
46 
47 
48 
49 DEA
50 registration
51 number
52 
53 
54 
55 Reason
56 for
57 inventory:
58 ANNUAL
59 Change
60 in
61 PIC**
62 
63 
64 
65 (circle
66 one)
67 Drug(s)
68 Newly
69 Scheduled
70 Acquired
71 Inventory
72 _
73 Initial
74 Inventory
75 
76 
77 
78  
79 
80 
81 
82 DATE
83 Inventory
84 Conducted:
85 ——._
86 /____
87 ds.
88 
89 
90 
91 TIME
92 Inventory
93 Conducted:
94 
95 
96 
97 Circle
98 one:
99 Opening
100 of
101 Business
102 Closing
103 of
104 Business
105 
106 
107 
108 SIGNATURE
109 of
110 RPh
111 performing
112 count
113 
114 
115 
116 PRINTED
117 NAME
118 of
119 Rph
120 performing
121 count
122 
123 
124 
125 **if
126 Change
127 in
128 PIC
129 (circle
130

In [167]:
auto_fill_dict ={

 37:'store',
 39:'Address',
 41:'City',
 44:'state',
 45:'zip',
 51:'number',

 82:'Date',

 91:'time',

 121 :'rph name'}

auto_fill_values ={}

In [169]:

n_boxes = len(d['level'])
for i in range(n_boxes):
    if (i in auto_fill_dict.keys()):
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        print(x,y,w,h)
        print(d['text'][i])
        coordinates = (x+w+50, y-h-10)
        print(coordinates)
        
        if (i == 82):
            auto_fill_values['DD'] = (x+w+800, y-h-10)
            auto_fill_values['MM'] = (x+w+1200, y-h-10)
            auto_fill_values['YY'] = (x+w+1600, y-h-10)
        elif (i == 91): 
            auto_fill_values['time'] = (x+w+800, y-h-10)
        else:
            auto_fill_values[auto_fill_dict[i]] = coordinates


662 853 27 37
#
(739, 806)
322 961 200 40
Address:
(572, 911)
330 1072 98 48
City.
(478, 1014)
1240 1067 130 59
Statte:
(1420, 998)
1604 1072 288 48
Zip:
(1942, 1014)
738 1180 182 41
number
(970, 1129)
154 1763 123 37
DATE
(327, 1716)
150 1983 121 37
TIME
(321, 1936)
991 2802 133 35
count
(1174, 2757)


In [170]:
auto_fill_values

{'store': (739, 806),
 'Address': (572, 911),
 'City': (478, 1014),
 'state': (1420, 998),
 'zip': (1942, 1014),
 'number': (970, 1129),
 'DD': (1077, 1716),
 'MM': (1477, 1716),
 'YY': (1877, 1716),
 'time': (1071, 1936),
 'rph name': (1174, 2757)}

In [171]:
#Currently I am using hard coded values , but will enhance it to use random values from name, address, location dictionaries


image_editable = ImageDraw.Draw(my_image)
for k,v in auto_fill_values.items():

    image_editable.text(v, k, (32, 0, 32), font=title_font)

In [172]:
my_image.save(r'C:\Users\prateek\Desktop\Ayoo docs\Code\paperless\auto_dataset\result2.jpg')